# Demo: Dual Y-Axis Chart Implementation

Este notebook demonstra a implementação da função `create_dual_y_axis_chart` que foi integrada ao dashboard.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import numpy as np

## Criação de Dados de Exemplo

In [ ]:
# Criar dados de exemplo similares aos do dashboard
dates = pd.date_range(start='2024-01-01', periods=12, freq='M')
np.random.seed(42)

# Simular volumes removidos (com alguns valores zero para realismo)
volumes_sao = np.random.randint(10, 100, 12)
volumes_sao[2] = 0  # Simular um mês sem remoção
volumes_sao[7] = 0  # Simular outro mês sem remoção

volumes_bailer = np.random.randint(5, 50, 12)
volumes_bailer[1] = 0
volumes_bailer[5] = 0

df_exemplo = pd.DataFrame({
    'Data': dates,
    'Volume Removido SAO (L)': volumes_sao,
    'Volume Removido Bailer (L)': volumes_bailer
})

print("Dados de exemplo:")
print(df_exemplo.head(10))

## Implementação da Função Dual Y-Axis

In [ ]:
def create_dual_y_axis_chart(df, x_col, y_col, title_prefix, color_primary="#156082", color_secondary="#c44d15"):
    """
    Cria um gráfico com duplo eixo Y.
    - Eixo Y esquerdo: valores originais (barras)
    - Eixo Y direito: valores acumulados (linha)
    
    Args:
        df: DataFrame com os dados
        x_col: nome da coluna do eixo X (geralmente 'Data')
        y_col: nome da coluna do eixo Y (ex: 'Volume Removido SAO (L)')
        title_prefix: prefixo do título (ex: 'Volume Removido SAO')
        color_primary: cor das barras (eixo esquerdo)
        color_secondary: cor da linha (eixo direito)
    
    Returns:
        fig: figura do Plotly com duplo eixo Y
    """
    
    # Ordenar por data para calcular acumulado corretamente
    df_sorted = df.sort_values(by=x_col).copy()
    
    # Calcular valores acumulados
    df_sorted[f'{y_col}_Acumulado'] = df_sorted[y_col].cumsum()
    
    # Criar gráfico base com barras no eixo Y primário
    fig = px.bar(
        df_sorted,
        x=x_col,
        y=y_col,
        title=f'{title_prefix} - Valores Individuais vs Acumulados',
        color_discrete_sequence=[color_primary]
    )
    
    # Adicionar linha com valores acumulados no eixo Y secundário
    fig.add_trace(
        go.Scatter(
            x=df_sorted[x_col],
            y=df_sorted[f'{y_col}_Acumulado'],
            mode='lines+markers',
            name=f'{title_prefix} Acumulado',
            yaxis="y2",
            line=dict(color=color_secondary, width=3),
            marker=dict(size=6)
        )
    )
    
    # Configurar layout com duplo eixo Y
    fig.update_layout(
        yaxis=dict(
            title=f"{title_prefix} (Individual)",
            side="left"
        ),
        yaxis2=dict(
            title=f"{title_prefix} (Acumulado)", 
            overlaying="y",
            side="right"
        ),
        dragmode='zoom',
        xaxis_tickangle=-45,
        legend_title_text='Séries',
        hovermode='x unified'
    )
    
    # Atualizar nome da primeira trace (barras)
    fig.data[0].name = f'{title_prefix} Individual'
    
    return fig

## Teste 1: Volume Removido SAO

In [ ]:
# Criar gráfico dual y-axis para Volume SAO
fig_sao = create_dual_y_axis_chart(
    df_exemplo,
    x_col='Data',
    y_col='Volume Removido SAO (L)',
    title_prefix='Volume Removido SAO',
    color_primary='#156082',
    color_secondary='#c44d15'
)

fig_sao.show()

## Teste 2: Volume Removido Bailer

In [ ]:
# Criar gráfico dual y-axis para Volume Bailer
fig_bailer = create_dual_y_axis_chart(
    df_exemplo,
    x_col='Data',
    y_col='Volume Removido Bailer (L)',
    title_prefix='Volume Removido Bailer',
    color_primary='#e17b7b',
    color_secondary='#c44d15'
)

fig_bailer.show()

## Verificação dos Dados Acumulados

In [ ]:
# Verificar se os cálculos estão corretos
df_test = df_exemplo.copy()
df_test['SAO_Acumulado'] = df_test['Volume Removido SAO (L)'].cumsum()
df_test['Bailer_Acumulado'] = df_test['Volume Removido Bailer (L)'].cumsum()

print("Verificação dos valores acumulados:")
print(df_test[['Data', 'Volume Removido SAO (L)', 'SAO_Acumulado', 
               'Volume Removido Bailer (L)', 'Bailer_Acumulado']].head(10))

## Como a Função foi Integrada ao Dashboard

A função `create_dual_y_axis_chart` foi integrada ao dashboard EPA da seguinte forma:

### 1. **Novas Opções no Multiselect**
- **Volume Produto**: `'Dual Y-Axis SAO'`, `'Dual Y-Axis Bailer'`
- **Volume Bombeado**: `'Dual Y-Axis Bombeado'`

### 2. **Funcionalidade**
- **Eixo Y Esquerdo (Barras)**: Valores individuais para cada período
- **Eixo Y Direito (Linha)**: Valores acumulados ao longo do tempo
- **Hover Unificado**: Informações de ambos os eixos no mesmo tooltip
- **Zoom Habilitado**: Para análise detalhada de períodos específicos

### 3. **Cores Personalizadas**
- **SAO**: Primário `#156082` (azul), Secundário `#c44d15` (laranja)
- **Bailer**: Primário `#e17b7b` (vermelho claro), Secundário `#c44d15` (laranja)
- **Bombeado**: Primário `#156082` (azul), Secundário `#c44d15` (laranja)

### 4. **Layout Inteligente**
O dashboard agora suporta exibição flexível de 1-5+ gráficos com layout responsivo.